In [1]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import cv2                  # working with, mainly resizing, images
import numpy as np          # dealing with arrays


TEST_DIR = '/home/shreyas/PycharmProjects/After10:55/Test Set'
IMG_SIZE = 150
LR = 1e-4

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet)

model.load('my_model.tflearn')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from /home/shreyas/PycharmProjects/FashionDxSubmission/my_model.tflearn


In [66]:
train_data = np.load('train_data.npy')
train = train_data[75:]
test = train_data[:75]
X1 = np.zeros((770, 150, 150))
for index, i in enumerate(train):
    X1[index] = i[0]
X1 = X1.reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
X1 = X1 / 255

In [67]:
test_x = np.zeros((75, 150, 150))

for index, i in enumerate(test):
    test_x[index] = i[0]

test_x = test_x.reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_x = test_x / 255
test_y = [i[1] for i in test]

In [68]:
from tqdm import tqdm  
import os
import cv2                  # working with, mainly resizing, images
import numpy as np          # dealing with arrays
from random import shuffle 

TEST_DIR = '/home/shreyas/PycharmProjects/After10:55/Test Set'
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[1]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data
test_data = process_test_data()

100%|██████████| 38/38 [00:01<00:00,  9.45it/s]


In [69]:
predictions = []
score = model.predict(test_x)
for i in range(75):
    if np.argmax(score[i]) == 1:
        predictions.append([0, 1, 0])
    elif np.argmax(score[i]) == 2: 
        predictions.append([0, 0, 1])
    else:
        predictions.append([1, 0, 0])
print(predictions)

[[0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [70]:
predictions_arg = []
for i in range(75):
    if np.argmax(score[i]) == 1:
        predictions_arg.append(1)
    elif np.argmax(score[i]) == 2: 
        predictions_arg.append(2)
    else:
        predictions_arg.append(0)
print(predictions_arg)

[2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 2, 2, 1, 0, 2, 1, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 1]


In [71]:
test_y_arg = []
for i in range(75):
    if np.argmax(test_y[i]) == 1:
        test_y_arg.append(1)
    elif np.argmax(test_y[i]) == 2: 
        test_y_arg.append(2)
    else:
        test_y_arg.append(0)
print(test_y_arg)

[2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 0, 1, 2, 2, 2, 0, 2, 0, 0, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 1]


In [78]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [72]:
print("Precision: {}%".format(100*precision_score(test_y_arg, predictions_arg, average="weighted")))

Precision: 83.09167367535744%


In [73]:
print("Recall: {}%".format(100*recall_score(test_y_arg, predictions_arg, average="weighted")))
print ("f1_score: {}%".format(100*f1_score(test_y_arg, predictions_arg, average="weighted")))

Recall: 81.33333333333333%
f1_score: 81.21102921102921%


In [77]:
matrix = confusion_matrix(test_y_arg, predictions_arg)
print(matrix)

[[30  0  3]
 [ 0  5  0]
 [11  0 26]]


In [80]:
accuracy = accuracy_score(test_y_arg, predictions_arg)
print(accuracy)

0.8133333333333334
